In [22]:
import pandas as pd
import numpy as np
import math
import geopandas as gpd
from shapely.geometry import Point

# Clip zip code shapefile to only SF

In [23]:
communities = gpd.read_file('../raw_data/zip_codes/geo_export_37348335-ac06-4ade-bf6d-453d393d539d.shp')

In [24]:
communities = communities[communities["po_name"] == "SAN FRANCISCO"]

In [60]:
communities.to_file('../clean_data/SF_zip_codes/SF_zip_codes.shp')

# Clean parks data

In [25]:
parks = pd.read_csv('../raw_data/parks_data.csv')

In [26]:
parks = parks.iloc[1:,:]
parks = parks.dropna(subset = ["Location 1"])

In [27]:
temp = parks["Location 1"].apply(lambda x: x.split('\n'))
parks["address"] = temp.apply(lambda x: x[0])
parks["city_state"] = temp.apply(lambda x: x[1])
parks["lat_long"] = temp.apply(lambda x: x[2])
del parks["Location 1"]
del parks["Lat"]

# Add zip code field to PARKS data using spatial join

In [28]:
parks["geometry"] = parks["lat_long"].apply(lambda x: Point(tuple(reversed(eval(x)))))
parks.crs = {'init': 'epsg:4326'}

In [29]:
parks = gpd.sjoin(parks, communities, how="inner", op='intersects')
del parks["index_right"]

In [63]:
parks.to_csv('../clean_data/parks.csv', index=False)

# Clean schools data

In [48]:
schools = pd.read_csv('../raw_data/Schools.csv')

In [49]:
schools = schools.dropna(subset = ["Location 1"])
temp = schools["Location 1"].apply(lambda x: x.split('\n'))
schools["State"] = temp.apply(lambda x: x[0])
schools["lat_long"] = temp.apply(lambda x: x[1])
del schools["Location 1"]

# Add zip code field to SCHOOLS data using spatial join

In [50]:
schools["geometry"] = schools["lat_long"].apply(lambda x: Point(tuple(reversed(eval(x)))))
schools.crs = {'init': 'epsg:4326'}

In [51]:
schools = gpd.sjoin(schools, communities, how="inner", op='intersects')
del schools["index_right"]
del schools["State"]

In [65]:
schools.to_csv('../clean_data/schools.csv', index=False)

# Add zip code field to BART STATIONS data using spatial join

In [53]:
bart_stations = gpd.read_file('../raw_data/bart_stations/BART_Sta_13.shp')

In [56]:
bart_stations = bart_stations.to_crs({'init': 'epsg:4326'})

In [57]:
bart_stations = gpd.sjoin(bart_stations, communities, how="inner", op='intersects')
del bart_stations["index_right"]

In [67]:
bart_stations.to_csv('../clean_data/bart_stations.csv', index=False)